In [1]:
%cd /content/drive/MyDrive/Aerial_Scene_Recognition

/content/drive/MyDrive/Aerial_Scene_Recognition


In [2]:
!pip install -r requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
current_directory = os.getcwd()

In [4]:
import torch
from models import imagebind_model
from models.imagebind_model import ModalityType

device = "cpu"

In [5]:
model = imagebind_model.imagebind_huge(pretrained=True)

In [6]:
model.eval()
model.to(device)

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [7]:
loaded_inputs = torch.load('test_inputs.pth', map_location=torch.device('cpu'))

In [8]:
loaded_inputs['vision'].shape

torch.Size([1063, 3, 224, 224])

In [9]:
loaded_inputs['audio'].shape

torch.Size([1063, 3, 1, 128, 204])

In [10]:
import h5py

with h5py.File('test_vision_embeddings.h5', 'w') as h5f:
  num_embeddings = loaded_inputs['vision'].shape[0]
  embedding_size = 1024
  dataset = h5f.create_dataset('test_vision_embeddings', (num_embeddings, embedding_size), dtype='f')
  for i in range(num_embeddings):

        embd = model({'vision': torch.unsqueeze(loaded_inputs['vision'][i], dim=0)})

        dataset[i] = embd['vision'].detach().cpu().numpy()

        del embd

In [11]:
import h5py
with h5py.File('test_audio_embeddings.h5', 'w') as h5f:
  num_embeddings = loaded_inputs['audio'].shape[0]
  embedding_size = 1024
  dataset = h5f.create_dataset('test_audio_embeddings', (num_embeddings, embedding_size), dtype='f')
  for i in range(num_embeddings):

        embd = model({'audio': torch.unsqueeze(loaded_inputs['audio'][i], dim=0)})

        dataset[i] = embd['audio'].detach().cpu().numpy()

        del embd